## Import

In [1]:
# create a Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TreeDocExam").getOrCreate()

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier 

## Load data

In [3]:
ls

 College.csv*             'Tree Methods Code Along.ipynb'*
 dog_food.csv*             Tree_Methods_Consulting_Project.ipynb*
 My_doc_example.ipynb*     Tree_Methods_Consulting_Project_SOLUTION.ipynb*
 sample_libsvm_data.txt*   Tree_Methods_Doc_Example.ipynb*


In [4]:
df = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [5]:
type(df)

pyspark.sql.dataframe.DataFrame

In [7]:
print(df.count(), len(df.columns))

100 2


In [8]:
df.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [9]:
train_data, test_data = df.randomSplit([0.7, 0.3])

## Training

In [14]:
# define a decision tree classifier
dtc = DecisionTreeClassifier(featuresCol = "features", labelCol="label")
# random forest clf
rfc = RandomForestClassifier(featuresCol = "features", labelCol="label", numTrees=100)
# gradient boosting
gbc = GBTClassifier(featuresCol="features", labelCol="label")

In [15]:
tree_model = dtc.fit(train_data)

In [16]:
forest = rfc.fit(train_data)

In [17]:
gb_model = gbc.fit(train_data)

## Predictions

In [18]:
# decision tree
tree_test_pred = tree_model.transform(test_data)

In [20]:
tree_test_pred.show(5)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[100,101,102...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[121,122,123...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,148...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 5 rows



In [21]:
# random forest
forest_test_pred = forest.transform(test_data)

In [22]:
forest_test_pred.show(5)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [98.0,2.0]|[0.98,0.02]|       0.0|
|  0.0|(692,[98,99,100,1...|   [98.0,2.0]|[0.98,0.02]|       0.0|
|  0.0|(692,[100,101,102...|  [52.0,48.0]|[0.52,0.48]|       0.0|
|  0.0|(692,[121,122,123...|   [96.0,4.0]|[0.96,0.04]|       0.0|
|  0.0|(692,[122,123,148...|   [91.0,9.0]|[0.91,0.09]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 5 rows



In [23]:
# gradient boosting
gb_test_pred = gb_model.transform(test_data)

In [24]:
gb_test_pred.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[98,99,100,1...|[-0.7282543865094...|[0.18900188013302...|       1.0|
|  0.0|(692,[100,101,102...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[121,122,123...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[122,123,148...|[1.54350200272498...|[0.95635347857270...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## Evaluation

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
acc_eval = MulticlassClassificationEvaluator(metricName="accuracy")
f1_eval = MulticlassClassificationEvaluator(metricName="f1")

In [30]:
# decision tree
tree_acc = acc_eval.evaluate(tree_test_pred)
print(tree_acc)
tree_f1 = f1_eval.evaluate(tree_test_pred)
print(tree_f1)

0.9655172413793104
0.965162206541517


In [35]:
# random forest
rf_acc = acc_eval.evaluate(forest_test_pred)
print(rf_acc)
rf_f1 = f1_eval.evaluate(forest_test_pred)
print(rf_f1)

1.0
1.0


In [36]:
# gradient boosting
gb_acc = acc_eval.evaluate(gb_test_pred)
print(gb_acc)
gb_f1 = f1_eval.evaluate(gb_test_pred)
print(gb_f1)

0.9655172413793104
0.965162206541517
